# Exercise 2: Multiple vector-valued Dirichlet BCs

## Contents

We will learn

 - How to impose Dirichlet boundary conditions only in selected components
 - How to impose Dirichlet boundary conditions described by more than one function

## Problem statement

Solve the linear elasticity problem defined on the 3D domain

<img src="solid.png" width="350">

with Dirichlet and Neumann BCs on the different coloured regions:

$$
\left\lbrace
\begin{aligned}
-∇\cdot\sigma(u) = 0 \ &\text{in} \ \Omega,\\
u = 0 \ &\text{on}\ \Gamma_{\rm Green},\\
u_1 = \delta \ &\text{on}\ \Gamma_{\rm Blue},\\
\sigma(u)\cdot \boldsymbol{n} = 0 \ &\text{on}\  \Gamma_{\rm Olive}.\\
\end{aligned}
\right.
$$

The stress tensor is defined as

$$
\sigma(u) \doteq \lambda\ {\rm tr}(\varepsilon(u)) \ \mathrm{Id} +2 \mu \  \varepsilon(u),
$$

We consider $E=70\cdot 10^9$ Pa and $\nu=0.33$.

Then $\lambda = (E\nu)/((1+\nu)(1-2\nu))$ and $\mu=E/(2(1+\nu))$.

## Numerical scheme

We consider a standard FEM.

Find $u\in U$ such that $a(u,v) = 0$ for all $v\in V$, where
$$
a(u,v)\doteq \int_{\Omega} \sigma(u) : \varepsilon(v) \ {\rm d}\Omega, \quad l(v) = 0.0.
$$

## Load and inspect the discrete model

### Exercise 1

_Create the discrete model from the Gmsh file `solid.msh`._

In [ ]:
# Have you already loaded all the packages you need?

### Exercise 2

_Write the model to vtk and open the resulting files with Paraview. Visualize the faces of the model and color them by each of the available fields. Identify the field names representing the boundaries $\Gamma_{\rm Blue}$ and $\Gamma_{\rm Green}$._

## Set up FE spaces and Dirichlet BCs

### Test FE space

We will construct the vector-valued test FE space as follows:

```
order = 1
reffe = ReferenceFE(lagrangian,VectorValue{3,Float64},order)
V = TestFESpace(model,reffe;
 conformity=:H1,
 dirichlet_tags=dirichlet_tags,
 dirichlet_masks=dirichlet_masks)
```

Vector-valued interpolation is selected via the option `valuetype=VectorValue{3,Float64}`.

We will next fill in
1. `dirichlet_tags` to identify the Dirichlet regions, and
2. `dirichlet_masks` to specify which components of the displacement are constrained.

### Exercise 3

_Fill in the `dirichlet_tags` with the tag names identified in **Exercise 2** for $\Gamma_{\rm Blue}$ and $\Gamma_{\rm Green}$._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

  The input format of `dirichlet_tags` is a 1D array of tag name strings `["tag_name_1",...,"tag_name_n"]`.
</details>

### Exercise 4

_Fill in the `dirichlet_masks` to select the displacement components to constrain._

We constrain only the first component on the boundary $\Gamma_{\rm Blue}$, whereas we constrain all components on $\Gamma_{\rm Green}$.

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

  The input format of `dirichlet_masks` is a 1D array with the same length and order as `dirichlet_tags`. Each entry of `dirichlet_masks` is a boolean tuple of length the problem dimension `D`, i.e. `(bool_x1,...,bool_xD)`. If `bool_xi == true`, then the `xi` component of the displacement is constrained, otherwise it is free. Beware of passing the `dirichlet_masks` in the same order as the `dirichlet_tags`.
</details>

We can now instantiate the vector-valued test FE space.

In [ ]:
order = 1
reffe = ReferenceFE(lagrangian,VectorValue{3,Float64},order)
V     = TestFESpace(model,reffe;
                    conformity=:H1,
                    dirichlet_tags=dirichlet_tags,
                    dirichlet_masks=dirichlet_masks)

### Trial FE space

The Dirichlet boundary conditions must be described with two different functions, one for boundary $\Gamma_{\rm Blue}$ and another one for $\Gamma_{\rm Green}$.

### Exercise 5

_Define the Dirichlet functions according to the problem statement._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

  The functions must be vector-valued with the format `VectorValue(val_x1,...,val_xD)`.
</details>

### Exercise 6

_Define the trial FE space `U`._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

  Pass the Dirichlet functions in a one-dimensional array, in the same order as `dirichlet_tags` and `dirichlet_masks`.
</details>

We will now visually check the Dirichlet values are being correctly assigned on the Dirichlet boundaries.

### Exercise 7

_Create a `BoundaryTriangulation` of the Dirichlet boundaries._

Next, we create a FE Function of `U` with zero-valued free values.

In [ ]:
vh = zero(U)

### Exercise 8

_Plot `vh` on the Dirichlet boundaries using `writevtk`_

The plot of the x-component of `vh` should look as follows.

<img src="fig_dbc.png" width="350">

## From weak form to visualising the solution

### Material parameter constants

In [ ]:
const E = 70.0e9
const ν = 0.33
const λ = (E*ν)/((1+ν)*(1-2*ν))
const μ = E/(2*(1+ν))

### Stress tensor

The symmetric gradient operator is represented by the function `ε` provided by Gridap (also available as `symmetric_gradient`). However, function `σ` representing the stress tensor is not predefined in the library and it has to be defined ad-hoc by the user.

### Exercise 9

_Define the linear stress tensor `σ`._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

  Function `σ` takes a strain tensor `ε` and computes the associated stress tensor using the Lamé constants.
</details>

### Integration mesh and measure

In [ ]:
degree = 2*order
Ω      = Interior(model)
dΩ     = Measure(Ω,degree)

### Exercise 10

_Write the weak form of the problem. Both the bilinear and the linear form._


<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

  Compose function `σ` with the strain field `ε(u)` in order to compute the stress field associated with the trial function `u`, i.e., `σ∘ε(u)`.
</details>

### Assemble and solve the problem

In [ ]:
op = AffineFEOperator(a,l,U,V)
uh = solve(op)

### Exercise 10

_Write the solution of the problem along with the strain and stress tensors._

If you plot the deformed displacement in ParaView, it can be clearly observed (see next figure) that the surface $\Gamma_{\rm Blue}$ is pulled in $x_1$-direction and that the solid deforms accordingly.

<img src="disp_ux_40.png" width="350">

### Bonus exercise

Modify this model to solve a multi-material linear elasticity problem as done [here](https://gridap.github.io/Tutorials/dev/pages/t003_elasticity/#Multi-material-problems-1).

**Exercise done!**

## References

This exercise is adapted from the [Gridap Tutorial 3: Linear elasticity](https://gridap.github.io/Tutorials/dev/pages/t003_elasticity/#Tutorial-3:-Linear-elasticity-1).

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*